In [1]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import time
import numpy as np

from joblib import Parallel, delayed
from skimage.io import imread

from canon.pattern import RescaleExtractor
from canon.util import split_workload

Using TensorFlow backend.


In [2]:
def process_img(img_files, shape):
    extractor = RescaleExtractor(shape)
    img_data = np.array([imread(f) for f in img_files])
    img_idx = np.array([[int(f[-9:-4])] for f in img_files])
    codes = extractor.features(img_data)
    return np.hstack([img_idx, codes])

def extract_features(jpg_dir, model_name):
    dir_path = os.path.join("img", jpg_dir)
    filenames = [os.path.join(dir_path, filename) for filename in os.listdir(dir_path)]
    print('Found %d files in the directory %s.' % (len(filenames), dir_path))

    fn_grps = split_workload(filenames, 32)

    with Parallel(n_jobs=-1, verbose=12) as parallel:
        data = parallel(delayed(process_img)(grp, model_name) for grp in fn_grps)
        data = np.vstack(data)
        data = data[data[:,0].argsort()]
        print("Loaded a data of shape {}".format(data.shape))
    
    return data

In [4]:
jpg_dir = "BTO_25C_wb3_60_100"
rescale_shape = (128, 128)
npy_suffix = "_rescale_%d_%d" % (rescale_shape[0], rescale_shape[1])
feature_file = jpg_dir + npy_suffix + ".npy"

print("Saving features to " + feature_file)
data = extract_features(jpg_dir, rescale_shape)
np.save("features/" + feature_file, data)

Saving features to BTO_25C_wb3_60_100_rescale_128_128.npy
Found 6000 files in the directory img/BTO_25C_wb3_60_100.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

ValueError: all the input arrays must have same number of dimensions